# Isotropic, isothermal linear elasticity UMAT Description

## Overview

UMAT for isotropic elasticity that incorporates, axisymmetric, plane and 3D stress formulations.

Isotropic elasticity formulation derived from Chapter 4, Introduction to Linear Elasticity, Gould, P. Axisymmetric and plane stress derived from EN2340 Computational Methods in Solid Mechanics, Allan Bower, Brown University. 

## Elastic Properties and Calculated Parameters

In this case the material is defined by its given Young's Modulus, $E$ and Poisson's ratio, $\nu$. The Young's modulus measures the tensile stiffness of a solid, this is achieved by quantifying the relationship between tensile stress $\sigma$ and axial strain $\varepsilon$ in the linear elastic region of a material and is given by the following formula;

\begin{equation}
E = \frac{\sigma}{\varepsilon}
\end{equation}

Poisson's ratio is a measure of material deformation in directions perpendicular to the direction of loading, given by the negative of the ratio of transverse strains, $\varepsilon_{trans}$ to axal strains, $\varepsilon_{axial}$  as shown in the following formula; 

\begin{equation}
    \nu = - \frac{\varepsilon_{trans}}{\varepsilon_{axial}}
\end{equation}

From these properties, Lamé's first parameter, $\mu$ and Lamé's second parameter, $\lambda$ are now defined; 

\begin{equation}
    \mu = \frac{E}{2+(1+\nu)}
\end{equation}

\begin{equation}
    \lambda = \frac{\nu E}{(1+\nu) (1-2 \nu)}
\end{equation}

In addition to these parameters for the plane stress case we will define the following parameter $\kappa$; 

\begin{equation}
    \kappa = \frac{E}{1-\nu^2}
\end{equation}

## 3D Case

### Theoretical Formulation

For homogeneous and isotropic materials Lamé's parameters define Hooke's law in three dimensions, where the stress tensor in index notation $\sigma_{ij}$ may be defined as;

\begin{equation}
    \sigma_{ij} = 2 \mu \varepsilon_{ij} + \lambda \delta_{ij} \varepsilon_{kk} 
\end{equation}

Where $\varepsilon_{ij}$ is the strain tensor in index notation and $\delta_{ij}$ is the the Kronecker delta function, this is simply a two variable function where the function is equal to one where the variables are equal and zero otherwise, this is expressed below; 

\begin{equation}
    \delta_{ij} = \begin{cases} 0 & \text{if} & i \neq j \\
    1 & \text{if} & i = j 
    \end{cases} 
\end{equation}

In order to implement a UMAT, we recall that the Jacobian or stiffness tensor $C$ must be defined, for linear elasticity because of small deformations the Jacobian is defined as;

\begin{equation}
    C = \frac{\partial \Delta \sigma}{\partial \Delta \varepsilon}
\end{equation}

Where $\Delta \sigma$ is the increment in stress and $\Delta \varepsilon$ is the increment in strain. This stiffness tensor $C$ is a fourth order tensor that forms part of the generalized form of Hooke's Law as seen below expressed in index notation;

\begin{equation}
    \sigma_{ij} = C_{ijkl} \varepsilon_{kl}
\end{equation}

For isotropic linear elastic materials the stiffness tensor in index notation $C_{ijkl}$ can be expressed in terms of Lamé's parameters as;

\begin{equation}
    C_{ijkl} = \lambda \delta_{ij} \delta_{kl} + \mu ( \delta_{ik} \delta_{jl} + \delta_{il} \delta_{jk}) 
\end{equation}

Where the stiffness tensor is written in the following form, shown in tensor notation, denoted by subscripts $ijkl$ and Voigt notation denoted by subscripts $\alpha \beta$ respectively;  

\begin{equation}
    \{C_{ijkl}\} =
    \begin{bmatrix}
        C_{1111} & C_{1122} & C_{1133} & C_{1123} & C_{1113} & C_{1112} \\
        C_{1122} & C_{2222} & C_{2233} & C_{2223} & C_{2213} & C_{2212} \\
        C_{1133} & C_{2233} & C_{3333} & C_{3323} & C_{3313} & C_{3312} \\
        C_{1123} & C_{2223} & C_{3323} & C_{2323} & C_{2313} & C_{2312} \\
        C_{1113} & C_{2213} & C_{3313} & C_{2313} & C_{2323} & C_{2313} \\
        C_{1112} & C_{2212} & C_{3312} & C_{2312} & C_{1312} & C_{1212} 
    \end{bmatrix}
\end{equation}

\begin{equation}
    \{C_{\alpha \beta}\} =
    \begin{bmatrix}
        C_{11} & C_{12} & C_{13} & C_{14} & C_{15} & C_{16} \\
        C_{12} & C_{22} & C_{23} & C_{24} & C_{25} & C_{26} \\
        C_{13} & C_{23} & C_{33} & C_{34} & C_{35} & C_{36} \\
        C_{14} & C_{24} & C_{34} & C_{44} & C_{45} & C_{46} \\
        C_{15} & C_{25} & C_{35} & C_{45} & C_{55} & C_{56} \\
        C_{16} & C_{26} & C_{36} & C_{46} & C_{56} & C_{66} \\
    \end{bmatrix}
\end{equation}

Following the index notation expression of the stiffness tensor, it can then be written in the matrix formulation below;

\begin{equation}
    \begin{bmatrix}
        \sigma_{11} \\
        \sigma_{22} \\
        \sigma_{33} \\
        \sigma_{12} \\
        \sigma_{23} \\
        \sigma_{31}
    \end{bmatrix}
    = 
    \begin{bmatrix}
        2 \mu + \lambda & \lambda & \lambda & 0 & 0 & 0 \\
        \lambda & 2 \mu + \lambda & \lambda & 0 & 0 & 0 \\
        \lambda & \lambda & 2 \mu + \lambda & 0 & 0 & 0 \\
        0 & 0 & 0 & 2 \mu & 0 & 0 \\
        0 & 0 & 0 & 0 & 2 \mu & 0 \\
        0 & 0 & 0 & 0 & 0 & 2 \mu
    \end{bmatrix}
    \begin{bmatrix}
        \varepsilon_{11} \\
        \varepsilon_{22} \\
        \varepsilon_{33} \\
        \varepsilon_{12} \\
        \varepsilon_{23} \\
        \varepsilon_{31}
    \end{bmatrix}
\end{equation}

### UMAT Implementation

This stress strain relationship is implemented in the UMAT using the following block of code;


```fortran
DO K1 = 1,NDI
	DO K2 = 1,NDI
		DDSDDE(K2,K1) = ELAMBDA
	END DO
	DDSDDE(K1,K1) = TWO*EMU+ELAMBDA
END DO
DO K1 = NDI+1,NTENS
    DDSDDE(K1,K1) = TWO*EMU
END DO
```

### Validation

To validate the implementation of 3D stress, a single element in uniaxial tension will be considered where the ABAQUS Standard, UMAT and theoretical results will be compared. The figure below shows the setup of a single element uniaxial tension model in ABAQUS with loads and boundary conditions.

![3DStressProblem](3DProblem.png)

The figure below illustrates stress / strain results in the direction of loading from ABAQUS Standard, UMAT and theoretical results. Note that all stress/strain results are normalized. 

![3DStressResults](3DResults.png)

It should be noted that this test alone is not adequate to verify or validate the result of a UMAT, it is recommended to compare the following load cases for displacement and load prescribed models; 
- Uniaxial
- Uniaxial in oblique direction
- Uniaxial with finite rotation
- Finite shear

## Axisymmetric Case

### Theoretical Formulation

We now consider an axisymmetric stress element as seen in the Figure below.

![axiStress](axiStress.png)

The stiffness matrix for the given case is as follows;

\begin{equation}
    \begin{bmatrix}
        \sigma_{11} \\
        \sigma_{22} \\
        \sigma_{33} \\
        \sigma_{12} \\
    \end{bmatrix}
    =
    \begin{bmatrix}
        1-\lambda & \lambda & \lambda & 0 \\
        \lambda & 1- \lambda & \lambda & 0 \\
        \lambda & \lambda & 1- \lambda & 0 \\
        0 & 0 & 0 & \mu 
    \end{bmatrix}
    \begin{bmatrix}
        \varepsilon_{11} \\
        \varepsilon_{22} \\
        \varepsilon_{33} \\
        \varepsilon_{12} \\
    \end{bmatrix}
\end{equation}

Where all symbols have their usual meanings.

### UMAT Implentation 

This stress strain relationship is implemented in the UMAT using the following block of code;


```fortran
DO K1 = 1,NDI
    DO K2 = 1,NDI
        DDSDDE(K2,K1) = ELAMBDA
	END DO
	DDSDDE(K1,K1) = ONE - ELAMBDA
END DO
DO K1 = NDI+1,NTENS
	DDSDDE(K1,K1) = EMU
END DO
```

### Validation 

The implementation of axisymmetric stress is validated by modeling a thick walled cylindrical vessel under internal pressure, ABAQUS Standard, UMAT and the theoretical calculations will all be compared. The ABAQUS model setup is shown below including the loading and boundary conditions. 

![axiProblem](axiProblem.png)

In this case the radial stress will be compared, the theoretical radial stress as a function of radius is given by the following equation

\begin{equation}
    \sigma_r = \frac{P_i R_i^2}{R^2_o-R^2_i} (1 - \frac{R_o^2}{r^2}) \; , R_i \leq r \leq R_o
\end{equation}

Where $P_i$ is the internal pressure, $R_i$ is the internal radius of the cylinder, $R_o$ is the external radius of the cylinder and $r$ is the radius variable which is bound between the internal and external radius. The values of these and relevant material properties are shown in the Table below;

| $$R_i(mm)$$ | $$R_o(mm)$$ | $$P_i(MPa)$$ | $$E(GPa)$$ | $$\nu$$ |
|-----------|-----------|------------|----------|-------|
| 60        | 96        | 15         | 200      | 0.3   |

The radial stress distribution along the radius of the cylinder is shown for the ABAQUS Standard, UMAT and theoretical calculations below;

![axiresults](axiResults.png)

## Plane Stress Case

### Theoretical Formulation

For plane stress we assume that a given stress element is thin enough that the stress in a principal direction is equal to zero. This is seen for the Figure below where we can infer that $\sigma_{zz} = \sigma_{yz} = \sigma_{xz} = 0$.


![planeStressBody](planeStressBody.png)

With these assumptions in mind the stress / strain relationship for a plane stress element can be defined as;

\begin{equation}
    \begin{bmatrix}
        \sigma_{11} \\
        \sigma_{22} \\
        \sigma_{12} \\
    \end{bmatrix}
    =
    \begin{bmatrix}
        \kappa & \kappa \nu & 0 \\
        \kappa \nu & \kappa & 0 \\
        0 & 0 & \frac{\kappa(1-\nu)}{2} 
    \end{bmatrix}
    \begin{bmatrix}
        \varepsilon_{11} \\
        \varepsilon_{22} \\
        \varepsilon_{12}
    \end{bmatrix}
\end{equation}

### UMAT Implementation

This stress strain relationship is implemented in the UMAT using the following block of code;


```fortran
DO K1 = 1,NDI
    DO K2 = 1,NDI
		DDSDDE(K2,K1) = EKAPPA*ENU
	END DO
	DDSDDE(K1,K1) = EKAPPA
	END DO
DO K1 = NDI+1,NTENS
	DDSDDE(K1,K1) = 0.5D0*EKAPPA*(ONE-ENU)
END DO
```

### Validation 

The implementation of plane stress is validated by comparison against the NAFEMS Benchmarks Linear Elastics Tests, LE1: Plane Stress Elements - Elliptic Membrane benchmark, a schematic of this model is shown in the figure below. 

![planeStress](planeStress.png)

The table below provides a comparison of the NAFEMS benchmark, ABAQUS Standard and UMAT for the maximum tangential stress $\sigma_{yy}$;

| $$\sigma_{yy} (MPa) NAFEMS$$ | $$\sigma_{yy} (MPa) Standard$$ | $$\sigma_{yy} (MPa) UMAT$$ |
|------------------------------|--------------------------------|----------------------------|
| 9.27                         | 9.28                           | 9.28                       |

